# Postprocessing of the Sentinel-hub statistical API results

This notebook demonstrates how to convert json-responses to pandas-compatible csv and how to to coombine data from cloud coverage runs with the regional statistics.

Converters and data handlers are given in [aapamires.py](aapamires.py)


In [2]:
import glob
import os
import json
import pandas as pd 
import aapamires

## Input data

In [3]:
g_small = glob.glob(r'./avo10ha_vkulkuiset_32635_shub_under1km2.gpkg/stats/2020/*.json')
g_large = glob.glob(r'./avo10ha_vkulkuiset_32635_shub_over1km2.gpkg/stats/2020/*.json')
stats_target  = r'./stats/2020'
src_clouds = r'./avo10ha_vkulkuiset_32635_shub_BoundingBoxes.gpkg/clouds/2020'
os.makedirs(stats_target,exist_ok=True)

In [4]:
# Convert each region to dataframe separately and save as csv
for f in g_small + g_large:
    with open(f, 'r') as fin:
        batch_result = json.load(fin)
    thisdf = aapamires.sumi_batch_response_to_df(batch_result)
    fname = os.path.splitext(os.path.basename(f))[0]
    target_csv = os.path.join(stats_target,fname+'.csv')  
    
    if thisdf is not None:
        # Find corrsponding BB cloud coverage data and merge it to the statistics
        g_cloud = glob.glob(os.path.join(src_clouds,os.path.split(f)[1]))
        cloudf = g_cloud[0]
        with open(cloudf, 'r') as cin:
            cloud_result = json.load(cin)
        cloud_df =  aapamires.sumi_clouds_response_to_df(cloud_result)
        bb_cloud = (cloud_df.loc[:, ['date', 'cloudcoverage']]).set_index('date').rename(columns={'cloudcoverage':'BBcloudcoverage'})
        thisdf = thisdf.set_index('date').join(bb_cloud)     
        thisdf.to_csv(target_csv)        
    else:
        with open(target_csv+'.failed','w') as f:
            f.write('Import failed!')
            f.write(json.dumps(batch_result, indent=4))        



Tried to import empty result!
Tried to import empty result!
